In [9]:
%load_ext autoreload
%autoreload 2

from agents import banditagents
from environments import bandits
import utils

import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
sns.set_theme(style="whitegrid", palette="pastel")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
configs_dir = '/Users/juliomartinez/Documents/PhD/socialmotivation/configs'
num_episodes = 50
solvers = ['EpsilonGreedyAgent']
#solvers = ['GreedyAgent']
#solvers = ['GreedyAgent', 'EpsilonGreedyAgent']
environment = 'TwoArmBandit'

bandit_config_filename = os.path.join(configs_dir, environment + '.yaml')
bandit_config = utils.get_config(bandit_config_filename)
env = bandits.Bandit(bandit_config)
num_iterations = 1000

demonstrator_all_rewards = np.empty(shape=(len(solvers),num_episodes, num_iterations),dtype=float)
demonstrator_all_actions = np.empty(shape=(len(solvers),num_episodes, num_iterations),dtype=float)
demonstrator_best_arm_estimate_proportions = np.arange(env.num_arms)

learner_all_rewards = np.empty(shape=(len(solvers),num_episodes, num_iterations),dtype=float)
learner_all_actions = np.empty(shape=(len(solvers),num_episodes, num_iterations),dtype=float)
learner_best_arm_estimate_proportions = np.arange(env.num_arms)

for i, solver in enumerate(solvers):
    
    # Get solver class 
    agentClass = getattr(banditagents, solver)
    
    # Get config file
    agents_config_filename = os.path.join(configs_dir, solver + '.yaml')
    agents_config = utils.get_config(agents_config_filename)

    for j in range(num_episodes):
        # run solver for demonstrator
        demonstrator = agentClass(agents_config['demonstrator'])   
        demonstrator(env)
        demonstrator_all_rewards[i,j,:] = demonstrator.reward_history
        demonstrator_all_actions[i,j,:] = demonstrator.arm_id_history
        # run solver for learner
        learner = agentClass(agents_config['learner'])
        learner(env, demonstrator)
        learner_all_rewards[i,j,:] = learner.reward_history
        learner_all_actions[i,j,:] = learner.arm_id_history
        
        # save best arm proportion
        #demonstrator_best_arm_estimate_proportions[demonstrator.best_arm_id]=+1
        #learner_best_arm_estimate_proportions[learner.best_arm_id]=+1
        
#demonstrator_best_arm_estimate_proportions = demonstrator_best_arm_estimate_proportions/np.sum(demonstrator_best_arm_estimate_proportions)
#learner_best_arm_estimate_proportions = learner_best_arm_estimate_proportions/np.sum(learner_best_arm_estimate_proportions)